In [169]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
import logic as lc

#Initialize pre requisites for flow generation
#Data Needed: TVC Summary, reference dict(TVC definitions to OSM definitions)
PATH = Path("Data\Intersection Data\Corrales - JR Borja\TVC Summary.xlsx")
DATA_PATH = f'{os.getcwd()}\Data\Intersection Data\Corrales - JR Borja'
reference_dict = lc.json_loader(os.path.join(DATA_PATH,'reference_dict.json'))

def format_tvc(PATH, DATA_PATH):
    reference_dict = lc.json_loader(os.path.join(DATA_PATH,'reference_dict.json'))
    #puts all sheets in one dataframe dictionary
    route_dfs = {}
    with pd.ExcelFile(PATH) as xls:
        sheets = xls.sheet_names
        for sheet in sheets:
            route_dfs[sheet] = pd.read_excel(xls, sheet)
            route_dfs[sheet].fillna(0,inplace=True) #fills null values with 0
            
    for df_title in route_dfs:
        df = route_dfs[df_title]

        #from time columns, adds when the entry starts and ends in seconds
        df['begin'] = np.arange(start=0, stop=int(len(df['Time'])*3600), step=3600)
        df['end'] = np.arange(start=3600, stop=int(len(df['Time'])*3600+1), step=3600)
        
        #from reference_dict, adds corresponding starting and endig lane column
        routes = df_title.split('_')
        df['from'] = [reference_dict[f'{routes[0]}_a']]*df.shape[0]
        df['to']=[reference_dict[f'{routes[1]}_f']]*df.shape[0]

        df.drop(columns=['Time','others'], inplace=True)    #deletes the Time and others column
        route_dfs[df_title] = route_dfs[df_title].loc[0:df.shape[0]-2]  #removes last 2 columns
    return route_dfs


route_dfs = format_tvc(PATH,DATA_PATH)


In [170]:
def flow_generator(route_dfs):
    """
    This is the flow generator function

    To use this, the TVC must be passed through the cleaning function to properly correctly format each dataframe
    to be used in the funcion.

    route_dfs -> cleaned TVC summary using the format_tvc() function
    """

    xml_list=[] #initialize list to store all names of generated xml files
    def float_to_int(val):
        if type(x)==float:
            return int(val)
        return val
    for vehicle in route_dfs[sheets[0]].columns.values[0:-5]: #lists all vehicle types
        vehicle_properties = lc.json_loader(Path(f'Data\\Vehicle Data\\{vehicle}.json')) #Loads vehicle data from vehicle data folder
        
        def vtype_xml_line(vehicle_properties):
            """
            Returns the vtype header
            """
            data_list = " ".join([f'{_property}="{vehicle_properties[_property]}"' for _property in vehicle_properties])
            line=f"<vType {data_list}/>"
            return line
        
        flow_list = [] #initialize list to store all flow commands
        count=0
        #for each route, input the from to start end number and type xml attributes and applend to the flow list array
        for route in route_dfs:
            df = route_dfs[route]
            new_df = pd.concat([df[[vehicle]], df.iloc[:,-4:]], axis=1)
            columns = new_df.columns.values
            columns[0] = "number"
            columns = np.hstack([np.array(["id"]),columns])
            for i in range(new_df.shape[0]):
                values = new_df.loc[i].values
                values[0]= int(values[0])
                values = np.hstack([np.array([f"{vehicle}{count}"]),values])
                columns_values_zip = " ".join([f'{column}="{value}"' for column, value in zip(columns, values)])
                flow_command = f'<flow type="{vehicle_properties["id"]}" {columns_values_zip}/>'
                flow_list.append([flow_command, values[2]])
                count+=1
        
        #initializes the route xml file of each vehicle type
        with open(f'Simulation_Environment\Main Route Simulation\{vehicle_properties["id"]}.xml','w') as f:
            f.write(f"""<?xml version="1.0" encoding="UTF-8"?>
            <routes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/routes_file.xsd">\n
            
            """)
        
            f.write(vtype_xml_line(vehicle_properties))
        
        flow_list = [string for string, key in sorted(flow_list, key= lambda x: x[1])]
        
        print('Done',vehicle)

        #writes each flow command to to the xml file
        with open(f'Simulation_Environment\Main Route Simulation\{vehicle_properties["id"]}.xml','a') as f:
            f.write("\n")
            for flow in flow_list:
                f.write(f"              {flow}\n")
            f.write("\n</routes>")
        
        xml_list.append(f'{vehicle_properties["id"]}.xml')

    #creates a text file listing all route files generated
    with open("xml_list.txt","w") as f:
        f.write(", ".join(xml_list))

        
    
flow_generator(route_dfs)

Done cars_SUV
Done motorcycles_scooters
Done taxi
Done buses
Done smallTrucks
Done largeTrucks
Done bicycle_tricycle
Done jeepney
Done motorela
